In [20]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import timeit
import re

In [21]:
def update_url(district_number):
    
    district = district_number

    url = f"https://www.opensecrets.org/races/summary?cycle=2020&id=TN{district}&spec=N"
    
    return url;

def split_title(title):
    title = title.get_text()
    title = (
    title.replace(' 2020 Race • OpenSecrets', '')
         .replace('District ', '')
         .split(' ')
        )
    return title

def get_dataframe(state, district, election_results):
    regex_candidate = r"(\w+\s\w+)"
    incumbent = "Incumbent"
    regex_party = r"(\W\w\W)"
    winner = "Winner"

    columns_extract= {'Candidate_name': regex_candidate,'Party':regex_party}
    columns_contains = {'Incumbent':incumbent, 'Winner':winner}

    for key, value in columns_extract.items():
        election_results[key] = election_results['Candidate'].str.extract(value, expand=True)

    for key, value in columns_contains.items():
        election_results[key] = election_results['Candidate'].str.contains(value)

    election_results['State'] = state 

    election_results['District'] = district

    election_results = election_results[['Candidate_name', 
                                         'Party', 
                                         'State',
                                         'District',
                                         'Incumbent', 
                                         'Winner', 
                                         'Raised', 
                                         'Spent']]
    return election_results

def generate_dataframe(response):
    soup = BS(response.text)
    soup.prettify()
    
    
    title = soup.find('title')
    title = split_title(title)
    state = (title[0])
    district = title[1]
    
    election_results = pd.read_html(str(soup.find('table')))[0]
    election_results = get_dataframe(state, district, election_results)
    return election_results

election_results = pd.DataFrame(columns=['Candidate_name', 
                                         'Party', 
                                         'State',
                                         'District',
                                         'Incumbent', 
                                         'Winner', 
                                         'Raised', 
                                         'Spent'])

In [39]:
URL_states = "https://worldpopulationreview.com/state-rankings/number-of-representatives-by-state"

response_states = requests.get(URL_states)

soup = BS(response_states.text)
table = soup.prettify()


states = pd.read_html(str(soup.find('table')))[0]
states


,State,Number of Representatives Seats,2021 Pop.
0,California,53,39613493
1,Texas,36,29730311
2,New York,27,19299981
3,Florida,27,21944577
4,Pennsylvania,18,12804123
5,Illinois,18,12569321
6,Ohio,16,11714618
7,Michigan,14,9992427
8,Georgia,14,10830007
9,North Carolina,13,10701022


In [ ]:
for i in range(1, numb_districts + 1):
    district = '{:0>2}'.format(i)
    URL = update_url(district)
    response = requests.get(URL)
    
    results = generate_dataframe(response)
    
    election_results = election_results.append(results)
   
election_results    